In [1]:
import src.algonauts.data_processors.image_transforms
from src.algonauts.models import model_loaders

In [2]:
experiment = 'vgg_imagenet'
batch_size = 300

In [3]:
platform = 'jupyter_notebook' #@param ['colab', 'paperspace'] {allow-input: true}

if platform == 'jupyter_notebook':
    challenge_data_dir = '../data/algonauts_2023_challenge_data'
    exp_output_dir = f'../data/out/{experiment}'

if platform == 'paperspace':
    challenge_data_dir = '/storage/algonauts_2023_challenge_data'
    exp_output_dir = f'/notebooks/data/out/{experiment}'

## Specify model and appropriate image preprocessing steps
Define a function to load and return the model, printing its layers.
Here we first load the model and print its nodes

### Choose to load a pretrained model from keras.applications

In [4]:
model_loader = lambda: model_loaders.load_vgg16()

### Or choose to load a model from file

In [ ]:
model_filename = 'model.h5'
transform_image = src.algonauts.data_processors.image_transforms.transform_alexnet
model_loader = lambda: model_loaders.load_from_file(model_filename, transform_image)

### Print the model layers

In [5]:
model, _ = model_loader()
print(*(layer.name for layer in model.layers), sep=' -> ')
del model

2023-03-24 13:24:46.004653: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-24 13:24:46.005366: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
input_1 -> block1_conv1 -> block1_conv2 -> block1_pool -> block2_conv1 -> block2_conv2 -> block2_pool -> block3_conv1 -> block3_conv2 -> block3_conv3 -> block3_pool -> block4_conv1 -> block4_conv2 -> block4_conv3 -> block4_pool -> block5_conv1 -> block5_conv2 -> block5_conv3 -> block5_pool


## Select layers and subjects
Now let's define which layer(s) we will pick from, and which subject(s) to process

In [6]:
layers = ['block5_pool']
subjects = [
    1, 
    # 2, 3, 4, 5, 6, 7, 8
    ]


## Run pipeline

In [7]:
from src.algonauts.pipelines.tf_pipeline import run_tf_pipeline

run_tf_pipeline(batch_size=batch_size, model_loader=model_loader, layers=layers, subjects=subjects,
                challenge_data_dir=challenge_data_dir,
                exp_output_dir=exp_output_dir)

Running for layer block1_pool
Running for subject 1
Training images: 9841
Test images: 159
LH training fMRI data shape:
(9841, 19004)
(Training stimulus images × LH vertices)

RH training fMRI data shape:
(9841, 20544)
(Training stimulus images × RH vertices)
Training images: 9841
Test images: 159
Training stimulus images: 8856

Validation stimulus images: 985

Test stimulus images: 159
Loading datasets...
Found 9841 files belonging to 1 classes.
Using 8857 files for training.
Using 984 files for validation.
Found 159 files belonging to 1 classes.
Datasets loaded
Training Scaler


  0%|          | 0/30 [00:00<?, ?it/s]2023-03-24 01:00:31.528714: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-24 01:00:32.943508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 30/30 [02:13<00:00,  4.45s/it]


Scaler training finished
Training PCA


100%|██████████| 30/30 [5:37:32<00:00, 675.07s/it]   


PCA training finished
Extracting and transforming features...
Features extracted and transformed
Fitting regression...


ValueError: Found input variables with inconsistent numbers of samples: [8857, 8856]

## Print best layers

In [38]:
import json
from src.algonauts.evaluators.correlations import find_best_correlations

subj = 1
result = find_best_correlations(f'{exp_output_dir}/results.json', subj)
print(json.dumps(result, indent=2))


{
  "LH": {
    "V1v": {
      "layer": "block2_pool",
      "value": 0.5905791030486138
    },
    "V1d": {
      "layer": "block2_pool",
      "value": 0.5990486552463073
    },
    "V2v": {
      "layer": "block3_pool",
      "value": 0.5282678641036374
    },
    "V2d": {
      "layer": "block3_pool",
      "value": 0.5175909098720242
    },
    "V3v": {
      "layer": "block3_pool",
      "value": 0.5141076787726454
    },
    "V3d": {
      "layer": "block4_pool",
      "value": 0.5065629532466085
    },
    "hV4": {
      "layer": "block4_pool",
      "value": 0.44591214364383464
    },
    "EBA": {
      "layer": "block5_pool",
      "value": 0.4846002619845918
    },
    "FBA-1": {
      "layer": "block5_pool",
      "value": 0.4077181141971304
    },
    "FBA-2": {
      "layer": null,
      "value": null
    },
    "mTL-bodies": {
      "layer": null,
      "value": null
    },
    "OFA": {
      "layer": "block4_pool",
      "value": 0.43446271378361234
    },
    "FFA-1": 